## Problem Set 6: Predictive Modeling

### Name: Gabriella Rivera

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import LogisticRegression

#### 1 Predict Airbnb Price 

1.

In [2]:
airbnb = pd.read_csv('airbnb-bangkok-listings.csv.bz2', sep=',', usecols=['bedrooms', 'price', 'accommodates', 'room_type'])
airbnb.head()

,room_type,accommodates,bedrooms,price
0,Entire home/apt,3,1.0,"$1,845.00"
1,Private room,2,1.0,"$1,275.00"
2,Private room,2,1.0,$800.00
3,Private room,2,1.0,$800.00
4,Private room,2,1.0,"$1,845.00"


In [3]:
airbnb['price'] = airbnb['price'].str.replace('$', '')  # Remove dollar sign
airbnb['price'] = airbnb['price'].str.replace(',', '')  # Remove commas

airbnb['price'] = pd.to_numeric(airbnb['price'])

In [5]:
clean_airbnb = airbnb.dropna(subset=['bedrooms', 'price', 'accommodates', 'room_type'])

In [6]:
clean_airbnb.isna().any()

room_type       False
accommodates    False
bedrooms        False
price           False
dtype: bool

2.

In [8]:
clean_airbnb['bedrooms'] = clean_airbnb['bedrooms'].astype(int)
clean_airbnb.loc[clean_airbnb['bedrooms'] >= 4, 'bedrooms'] = 4

/var/folders/mj/sjlj3tpn0j799b38ldhrgyqh0000gn/T/ipykernel_63383/2925388353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_airbnb['bedrooms'] = clean_airbnb['bedrooms'].astype(int)


In [13]:
m = smf.ols('np.log(price) ~ bedrooms + room_type + accommodates', data = clean_airbnb).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     945.5
Date:                Tue, 14 May 2024   Prob (F-statistic):               0.00
Time:                        20:54:32   Log-Likelihood:                -16582.
No. Observations:               15200   AIC:                         3.318e+04
Df Residuals:                   15194   BIC:                         3.322e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     6.5849      0.015    449.007      0.000       6.556       6.614
room_type[T.Hotel room]       0.1047      0.028      3.799      0.000       0.051       0.159
room_type[T.Private room]    -0.0856      0.013     -6.745      0.000      -0.110      -0.061
room_type[T.Shared room]     -0.7983      0.030    -26.701      0.000      -0.857      -0.740
bedrooms                      0.4171      0.012     35.224      0.000       0.394       0.440
accommodates                  0.0394      0.004     11.136      0.000       0.032       0.046
==============================================================================
Omnibus:                     4221.820   Durbin-Watson:                   1.636
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22128.885
Skew:                           1.241   Prob(JB):                         0.00
Kurtosis:                       8.365   Cond. No.                         22.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2.

In [19]:
phat = m.predict()
phat
# phat[:10]

array([7.12010689, 6.995152  , 6.995152  , ..., 7.6159347 , 7.08074826,
       7.12010689])